In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
data = pd.read_excel("ini.xlsx")

# 背景

- 一名员工加入后工作 3 至 15 天

- 每个车间人数为 5 至 15 人不等,并随员工的流动而不断变化

| 员工数量 |        运输专班安排        |
| :------: | :------------------------: |
|   5-7    |           班次A            |
|   8-10   |        班次A、班次B        |
|  11-12   |    班次A、班次B、班次C     |
|  13-15   | 班次A、班次B、班次C、班次D |

# 硬约束条件（必须满足）

- 每个专班需安排两名员工,其中至少一名需具有驾驶资格。

- 每名员工每天至多安排一次专班。

# 软约束条件（尽可能满足）

- 使每位员工当前累计安排专班次数与当前工作天数之比和所有员工当前总累计安排专班次数与当前总工作天数之比的差值不超过0.1

- 每个班次至少有一名已有专班经验的员工。

- 每名员工在不同天内参与的专班在具体班次上应尽可能平均。

- 历史事故次数较高的员工参与专班时须与历史事故次数较低的员工组成专班。历史事故次数为 0 或 1 的可认为历史事故次数较低,历史事故次数大于 3 的可认为历史事故次数较高。
 

# 数学模型

有驾驶经验的员工集合用 $D$ 表示，没有驾驶经验的员工集合用 $S$ 表示，班次集合用 $K$ 表示。那么，可以用 $x_{dk}$ 表示第 $d$ 名有驾驶经验的员工在班次 $k$ 的排班安排,用 $y_{sk}$ 表示第 $s$ 名没有驾驶经验的员工在班次 $k$ 的排班安排。

- $1\leq\sum_{d\in D}x_{dk}\leq 2, \forall k\in K$

- $\sum_{d\in D}x_{dk}+\sum_{s\in S}y_{sk}=2, \forall k\in K$

- $\sum_{k\in K}x_{dk}\leq1,\forall d\in D$

- $\sum_{k\in K}y_{sk}\leq1,\forall s\in S$

对于软约束条件,$p_1,p_2,p_3,p_4$ 表示相应的惩罚权重值，$s_1,s_2,s_3,s_4$ 表示所得解对软约束条件的违反次数。

In [2]:
NumOfPeople = data.shape[0]
if NumOfPeople >= 5 and NumOfPeople <= 7:
    NumOfClasses = 1
elif NumOfPeople >= 8 and NumOfPeople <= 10:
    NumOfClasses = 2
elif NumOfPeople >= 11 and NumOfPeople <= 12:
    NumOfClasses = 3
elif NumOfPeople >= 13 and NumOfPeople <= 15:
    NumOfClasses = 4

In [3]:
totalWorkDays = 0
totalWorkClasses = 0
for i in range(data.shape[0]):
    totalWorkDays += data.loc[:,"已工作天数"][i]
    totalWorkClasses += data.loc[:,"已安排A"][i]
    totalWorkClasses += data.loc[:,"已安排B"][i]
    totalWorkClasses += data.loc[:,"已安排C"][i]
    totalWorkClasses += data.loc[:,"已安排D"][i]
ratio = totalWorkClasses/totalWorkDays